In [1]:
from cassandra.cluster import Cluster
from datetime import datetime

# ==========================================
# 1️⃣ เชื่อมต่อ Cassandra
# ==========================================
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_stock
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('data_stock')

session.execute("""
        CREATE TABLE IF NOT EXISTS stock_price (
            symbol TEXT,
            time TIMESTAMP,
            open_price DOUBLE,
            high_price DOUBLE,
            low_price DOUBLE,
            close_price DOUBLE,
            volume BIGINT,
            value DOUBLE,
            PRIMARY KEY (symbol, time)
        ) WITH CLUSTERING ORDER BY (time DESC)
    """)
    
print("✅ Keyspace และ Table พร้อมใช้งาน!")

# ==========================================
# 2️⃣ ฟังก์ชันบันทึก DataFrame ลง Cassandra
# ==========================================
def insert_multiple_stocks(df, symbols):
    for symbol in symbols:
        print(f"\n📦 กำลังบันทึกข้อมูล {symbol} ...")
        # ตรวจ column ของหุ้นนั้น
        col_open = f"Open_{symbol}"
        col_high = f"High_{symbol}"
        col_low = f"Low_{symbol}"
        col_close = f"Close_{symbol}"
        col_volume = f"Volume_{symbol}"
        
        if col_open not in df.columns:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
            continue
        
        for idx, row in df.iterrows():
            timestamp = pd.Timestamp(row["time"]).to_pydatetime()
            session.execute("""
                INSERT INTO stock_price (
                    symbol, time, open_price, high_price, low_price,
                    close_price, volume, value
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,
                timestamp,
                float(row[col_open]),
                float(row[col_high]),
                float(row[col_low]),
                float(row[col_close]),
                int(row[col_volume]),
                float(row[col_close] * row[col_volume])
            ))
        print(f"✅ เพิ่มข้อมูล {len(df)} รายการของหุ้น {symbol} สำเร็จ!")

# ==========================================
# 3️⃣ เรียกใช้งานฟังก์ชัน
# ==========================================
insert_multiple_stocks(df, symbols)


✅ Keyspace และ Table พร้อมใช้งาน!


NameError: name 'df' is not defined

In [ ]:
# ==========================================
# 4️⃣ แสดงข้อมูลล่าสุดจาก Cassandra
# ==========================================
def show_latest_data(symbols, limit=5):
    for symbol in symbols:
        print(f"\n📊 ข้อมูลล่าสุดของ {symbol}:")
        
        # ดึง limit แถวล่าสุดเรียงตามเวลา
        rows = session.execute(f"""
            SELECT * FROM stock_price 
            WHERE symbol='{symbol}' 
            ORDER BY time DESC 
            LIMIT {limit}
        """)
        
        # แสดงผล
        for row in rows:
            print(f"เวลา: {row.time}, เปิด: {row.open_price}, สูง: {row.high_price}, ต่ำ: {row.low_price}, ปิด: {row.close_price}, ปริมาณ: {row.volume}, มูลค่า: {row.value}")

# เรียกใช้งานฟังก์ชัน
show_latest_data(symbols, limit=5)



📊 ข้อมูลล่าสุดของ AOT.BK:
เวลา: 2025-08-05 00:00:00, เปิด: 71.5, สูง: 72.0, ต่ำ: 71.0, ปิด: 71.8, ปริมาณ: 980000, มูลค่า: 70364000.0
เวลา: 2025-08-04 00:00:00, เปิด: 72.0, สูง: 72.5, ต่ำ: 71.5, ปิด: 72.2, ปริมาณ: 1200000, มูลค่า: 86640000.0
เวลา: 2025-08-03 00:00:00, เปิด: 70.8, สูง: 71.0, ต่ำ: 70.5, ปิด: 70.9, ปริมาณ: 950000, มูลค่า: 67355000.0
เวลา: 2025-08-02 00:00:00, เปิด: 71.0, สูง: 71.5, ต่ำ: 70.8, ปิด: 71.2, ปริมาณ: 1100000, มูลค่า: 78320000.0
เวลา: 2025-08-01 00:00:00, เปิด: 70.5, สูง: 71.2, ต่ำ: 70.3, ปิด: 71.0, ปริมาณ: 1000000, มูลค่า: 71000000.0
